### Расстояние Левенштейна

<img src ='https://wikimedia.org/api/rest_v1/media/math/render/svg/bb9db0430ed76b6826156bbafc02d1d0183eec8c'/>

m(a, b) = 0 если a = b и = 1 иначе

#### Алгоритм

для всех i от 0 до M

   для всех j от 0 до N
   
     вычислить D(i, j)
     
 вернуть D(M, N)

In [25]:
import numpy as np
from nltk import ngrams

In [2]:
def levenshtein_distance(str1, str2):
    m, n = len(str1), len(str2)
    d1 = np.arange(m + 1)[np.newaxis]
    d2 = np.arange(1, n + 1)[:, np.newaxis]
    d3 = np.zeros((n, m))
    d = np.vstack((d1, np.hstack((d2, d3))))
    for i in range(m):
        for j in range(n):
            d[j + 1, i + 1] = np.min([d[j, i + 1] + 1, d[j + 1, i] + 1, d[j, i] + int(str1[i] != str2[j])])
    return d # [n, m]

In [45]:
matrix = levenshtein_distance('Saturday', 'Sunday')

In [46]:
matrix

array([[0., 1., 2., 3., 4., 5., 6., 7., 8.],
       [1., 0., 1., 2., 3., 4., 5., 6., 7.],
       [2., 1., 1., 2., 2., 3., 4., 5., 6.],
       [3., 2., 2., 2., 3., 3., 4., 5., 6.],
       [4., 3., 3., 3., 3., 4., 3., 4., 5.],
       [5., 4., 3., 4., 4., 4., 4., 3., 4.],
       [6., 5., 4., 4., 5., 5., 5., 4., 3.]])

In [50]:
x, y = len('Sunday'), len('Saturday') 

In [51]:
np.argmin([matrix[x - 1, y], matrix[x, y - 1], matrix[x - 1, y - 1]])

2

In [10]:
matrix[x-1, y-1]

3.0

In [54]:
str1 = 'mum'
str2 = 'mamm'
str1 = "".join(list(map(lambda x: "".join(x), ngrams('^' + str1 + '_', 2))))
str2 = "".join(list(map(lambda x: "".join(x), ngrams('^' + str2 + '_', 2))))
str1, str2

('^mmuumm_', '^mmaammmm_')

In [55]:
print(levenshtein_distance(str1, str2))

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.]
 [ 1.  0.  1.  2.  3.  4.  5.  6.  7.]
 [ 2.  1.  0.  1.  2.  3.  4.  5.  6.]
 [ 3.  2.  1.  0.  1.  2.  3.  4.  5.]
 [ 4.  3.  2.  1.  1.  2.  3.  4.  5.]
 [ 5.  4.  3.  2.  2.  2.  3.  4.  5.]
 [ 6.  5.  4.  3.  3.  3.  2.  3.  4.]
 [ 7.  6.  5.  4.  4.  4.  3.  2.  3.]
 [ 8.  7.  6.  5.  5.  5.  4.  3.  3.]
 [ 9.  8.  7.  6.  6.  6.  5.  4.  4.]
 [10.  9.  8.  7.  7.  7.  6.  5.  4.]]


In [57]:
list(range(1, -1, -1))

[1, 0]

In [116]:
str1 = 'aumday'
str2 = 'sunday'
right = "".join(list(map(lambda x: "".join(x), ngrams('^' + str1 + '_', 2))))
wrong = "".join(list(map(lambda x: "".join(x), ngrams('^' + str2 + '_', 2))))
right, wrong

('^aauummddaayy_', '^ssuunnddaayy_')

In [117]:
matrix = levenshtein_distance(right, wrong)

In [118]:
from collections import defaultdict
d = defaultdict(int)

In [119]:
x, y = len(wrong), len(right)
while x not in [0, 1] and y not in [0, 1]:
    right_bigram, wrong_bigram = np.array((2, 0), dtype=str), np.array((2, 0), dtype=str)
    for i in range(1, -1, -1):  # we have even amount of characters
        paths = np.array([matrix[x - 1, y], matrix[x, y - 1], matrix[x - 1, y - 1]])
        # states: 0, 1, 2
        next_pos = np.argmin(paths)
        if matrix[x, y] == paths[next_pos]:  # there is no error
            right_bigram[i] = right[y - 1]
            wrong_bigram[i] = wrong[x - 1]
            if next_pos == 0:
                x -= 1
            elif next_pos == 1:
                y -= 1
            elif next_pos == 2:
                x -= 1
                y -= 1
            continue
        # delete
        if next_pos == 0:
            right_bigram[i] = '~'
            wrong_bigram[i] = wrong[x - 1]
            x -= 1
        # add
        elif next_pos == 1:
            right_bigram[i] = right[y - 1]
            wrong_bigram[i] = '~'
            y -= 1
        # change
        elif next_pos == 2:
            right_bigram[i] = right[y - 1]
            wrong_bigram[i] = wrong[x - 1]
            x -= 1
            y -= 1
    if x % 2 != 0:
        x -= 1
    if y % 2 != 0:
        y -= 1
    right_bigram = right_bigram[0] + right_bigram[1]
    wrong_bigram = wrong_bigram[0] + wrong_bigram[1]
    d[(right_bigram, wrong_bigram)] += 1

In [120]:
d

defaultdict(int,
            {('y_', 'y_'): 1,
             ('ay', 'ay'): 1,
             ('da', 'da'): 1,
             ('md', 'nd'): 1,
             ('um', 'un'): 1,
             ('au', 'su'): 1,
             ('^a', '^s'): 1})

In [69]:
a = np.array((2, 0), dtype=str)
a[0] = 's'
a[1] = 'a'

In [71]:
a[0] + a[1]

'sa'

In [60]:
import pandas as pd

In [75]:
df = pd.read_csv('queries_all.txt', sep='\t',  header=None)

In [78]:
df.head()

,0
0,видео секса смотреть бесплатно
1,шоплифтеры казань
2,платья 2016 фото новинки
3,вк
4,александр липовой


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991771 entries, 0 to 1991770
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
dtypes: object(1)
memory usage: 15.2+ MB


In [98]:
len(set('ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        'abcdefghijklmnopqrstuvwxyz'
        'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ'
        'абвгдеёжзийклмнопрстуфхцчшщьыъэюя'))

118

In [100]:
def split(line):
    split_line = []
    letters = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ'
                  'abcdefghijklmnopqrstuvwxyz'
                  'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ'
                  'абвгдеёжзийклмнопрстуфхцчшщьыъэюя')

    if not line:
        return split_line

    line = line.lower()

    # word consists of letters
    word = ''
    for c in line:
        if c in letters:
            word += c
        else:
            if word:
                split_line.append(word)
                word = ''
    return split_line

In [102]:
split('enrique iglesias duele el corazon скачать бесплатно')

['enrique', 'iglesias', 'duele', 'el', 'corazon', 'скачать']

In [106]:
for line in df.values:
    if '\t' in line[0]:
        print('yes')
        break

yes


In [107]:
line = 'enrique iglesias duele el corazon скачать бесплатно'

In [113]:
line.find('3')

-1

In [114]:
from collections import defaultdict

In [115]:
d = defaultdict(float)

In [116]:
d['d'] += 3

In [117]:
d['s'] += 4

In [118]:
len(d)

2

In [119]:
d

defaultdict(float, {'d': 3.0, 's': 4.0})

In [123]:
for i in d:
    d[i] += 1

In [124]:
d

defaultdict(float, {'d': 4.0, 's': 5.0})

In [125]:
d['d'] /= 3 * 2
d

defaultdict(float, {'d': 0.6666666666666666, 's': 5.0})